In [1]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
faq = pd.read_excel("../data/01_База_знаний.xlsx")
faq.head()

,Тема,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Что нельзя публиковать на RUTUBE,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
2,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за искус...,Монетизация на RUTUBE зависит в том числе от к...,МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,Почему могут отключить монетизацию на видео и ...,"Для каких статусов доступна монетизация, и поч...","Монетизацию на RUTUBE можно подключить, если в...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,Авторское право,Какой контент можно использовать для монетизац...,"То, что вы создали сами: видео, которое вы сня...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации


In [3]:
faq = faq.drop(['Тема', 'Классификатор 1 уровня', 'Классификатор 2 уровня'], axis=1)
faq.head()

,Вопрос из БЗ,Ответ из БЗ
0,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...
1,Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к..."
2,Почему могут отключить монетизацию из-за искус...,Монетизация на RUTUBE зависит в том числе от к...
3,"Для каких статусов доступна монетизация, и поч...","Монетизацию на RUTUBE можно подключить, если в..."
4,Какой контент можно использовать для монетизац...,"То, что вы создали сами: видео, которое вы сня..."


In [4]:
faq.to_csv('../data/knowledge_base.csv', index=False, encoding='utf-8')

In [5]:
import torch

torch.cuda.is_available()

True

In [6]:
from langchain_community.document_loaders import CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings

In [7]:
file_path = '../data/knowledge_base.csv'

embeddings = HuggingFaceEmbeddings(model_name='deepvk/USER-bge-m3', cache_folder='../cache/embedding/')

In [ ]:
raw_documents = CSVLoader(file_path, encoding='utf-8').load()

In [ ]:
raw_documents

In [ ]:
from langchain_community.vectorstores import FAISS

In [ ]:
db = FAISS.from_documents(raw_documents, embeddings)

In [ ]:
db.save_local('../cache/db/faiss_nikita/', 'faiss_nikita.db')

In [ ]:
db = FAISS.load_local(
    folder_path='../cache/db/faiss_nikita/',
    embeddings=embeddings,
    index_name='faiss_nikita.db',
    allow_dangerous_deserialization=True,
)

In [ ]:
test_retriever = db.as_retriever(
    # search_type='mmr',
    search_type='similarity',
    search_kwargs={
        'k': 3,
        # 'fetch_k': 5,
    }
)

In [ ]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(raw_documents, k=1)

In [ ]:
from joblib import dump, load

dump(bm25_retriever, '../cache/bm25.pkl')

In [ ]:
bm25_retriever: BM25Retriever = load('../cache/bm25.pkl')

In [ ]:
bm25_retriever

BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x000001B539929A50>, k=1)

In [ ]:
from langchain.retrievers import EnsembleRetriever

In [ ]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[test_retriever, bm25_retriever],
    weights=[0.9, 0.1],
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama.llms import OllamaLLM

template = """
Ты интеллектуальный помощник компании RUTUBE и ты очень точно отвечаешь на вопросы. Будь вежливым.
Ответь на вопрос, выбрав фрагмент из контекста, не меняя его по возможности, сохрани все имена, аббревиатуры, даты и ссылки. Вот контекст:

{context}

Вопрос: {question}

Если не можешь найти ответ в контексте, вежливо скажи, что не знаешь ответ
"""
prompt = ChatPromptTemplate.from_template(template)
model = OllamaLLM(model='gemma2:9b', base_url='http://26.251.162.207:11434')  # через vpn связь со своим ПК с GPU и развернутой LLM

# некоторые из моделей, которые мы пробовали
# mistral:7b -- so so
# llama3.1 -- so so
# wavecut/vikhr -- норм
# gemma2:9b -- норм

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
for a in chain.stream('Привет, кому принадлежат авторские права на контент?'):
    print(a, end = '')

Привет!  По умолчанию считается, что авторские права на размещённый на RUTUBE контент принадлежат тому пользователю, который его разместил. 
 Некоторые права передаются в рамках открытой лицензии третьим лицам, получившим доступ к этому контенту. Подробнее в разделе 4 Генерального пользовательского соглашения https://rutube.ru/info/agreement/ и в Приложении «Е» https://rutube.ru/info/ccl_agreement/.  


In [ ]:
print('\n\n'.join(d.page_content for d in ensemble_retriever.invoke('Кому принадлежат авторские права на контент, который я размещаю на платформе?')))

Вопрос из БЗ: Кому принадлежат авторские права на контент, который я размещаю на платформе?
Ответ из БЗ: По умолчанию считается, что авторские права на размещённый на RUTUBE контент принадлежат тому пользователю, который его разместил. Некоторые права передаются в рамках открытой лицензии третьим лицам, получившим доступ к этому контенту. Подробнее в разделе 4 Генерального пользовательского соглашения https://rutube.ru/info/agreement/ и в Приложении «Е» https://rutube.ru/info/ccl_agreement/.

Вопрос из БЗ: Кто такой правообладатель и как им стать?
Ответ из БЗ: Правообладателем контента может стать как его автор, так и любой другой человек или организация. Если вы хотите зарегистрироваться в качестве правообладателя и у вас есть документы, которые подтверждают ваши права на контент, напишите письмо на partners@rutube.ru. Правообладатель имеет право стать партнером и получать доход посредством монетизации.

Вопрос из БЗ: Могу ли я монетизировать видеоролики, в которых есть реакции на фил

In [ ]:
for a in chain.stream("Привет, как подключить монетизацию?"):
    print(a, end = '')

Привет!  Для подключения монетизации нужно загрузить на свой канал минимум 2 ролика и набрать 5000 просмотров начиная с 1 января 2021 года. После этого в разделе «Монетизация» в Студии RUTUBE появится анкета партнёра, которую нужно заполнить и подать заявку на монетизацию: https://studio.rutube.ru/monetization.

Подробнее о том, как это работает, можно прочитать здесь: https://rutube.ru/info/adv_oferta/. 




In [ ]:
for a in chain.stream("Как сварить кашу?"):
    print(a, end = '')

Я не могу ответить на ваш вопрос. Информация о приготовлении каши отсутствует в данном тексте.  




In [ ]:
for a in chain.stream("Как войти в приложении Студия RUTUBE?"):
    print(a, end = '')

Для входа в приложение Studio RUTUBE вы можете использовать свой номер телефона или адрес электронной почты. Если у вас возникли проблемы с входом, обратитесь за помощью к службе поддержки по адресу help@rutube.ru, указав ваш номер телефона или адрес электронной почты, использованные для регистрации аккаунта.

In [ ]:
def calculate_hit_rate(responses, ground_truth):
    hits = sum([1 for response, truth in zip(responses, ground_truth) if response == truth])

    return hits / len(ground_truth)

def calculate_mrr(responses, ground_truth):
    reciprocal_ranks = []
    for truth in ground_truth:
        if truth in responses:
            rank = responses.index(truth) + 1
            reciprocal_ranks.append(1 / rank)
        else:
            reciprocal_ranks.append(0)
    return sum(reciprocal_ranks) / len(ground_truth)

def calculate_relevancy(responses, query):
    relevant_responses = sum([1 for response in responses if query.lower() in response.lower()])
    return relevant_responses / len(responses)


In [ ]:
def context_precision(predictions, ground_truths):
    precision_scores = []
    for pred, truth in zip(predictions, ground_truths):
        relevant_items = set(truth.split())
        retrieved_items = set(pred.split())
        print(f'{relevant_items=}')
        print(f'{retrieved_items=}')
        if retrieved_items:
            precision_scores.append(len(relevant_items & retrieved_items) / len(retrieved_items))
        else:
            precision_scores.append(0)
    return sum(precision_scores) / len(precision_scores)

def faithfulness(predictions, ground_truths):
    return sum(1 for pred, truth in zip(predictions, ground_truths) if pred == truth) / len(predictions)

def answer_relevancy(predictions, ground_truths):
    relevancy_scores = []
    for pred, truth in zip(predictions, ground_truths):
        relevancy_scores.append(len(set(pred.split()) & set(truth.split())) / len(set(truth.split())) if truth else 0)
    return sum(relevancy_scores) / len(relevancy_scores)

In [10]:
data = pd.read_excel('../data/02_Реальные_кейсы.xlsx')
data.head()

,Вопрос пользователя,Ответ сотрудника,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Здравствуйте! Можно уточнить причины Правилhtt...,Добрый день!\nЧто нельзя публиковать на RUTUBE...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,"Добрый вечер, какой топ причин блокировки виде...",Добрый вечер!\nЧто заперщено публиковать на RU...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,Что запрещено в монетизации и что можно выклад...,"Здравствуйте!\nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Для монетизации можно использовать то, что вы ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации


In [11]:
data = data.drop(['Вопрос из БЗ', 'Ответ из БЗ', 'Классификатор 1 уровня', 'Классификатор 2 уровня'], axis=1)
data.head()

,Вопрос пользователя,Ответ сотрудника
0,Здравствуйте! Можно уточнить причины Правилhtt...,Добрый день!\nЧто нельзя публиковать на RUTUBE...
1,"Добрый вечер, какой топ причин блокировки виде...",Добрый вечер!\nЧто заперщено публиковать на RU...
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ..."
3,Что запрещено в монетизации и что можно выклад...,"Здравствуйте!\nМонетизация может отключиться, ..."
4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Для монетизации можно использовать то, что вы ..."


In [12]:
data.to_csv('../data/test_base.csv', index=False, encoding='utf-8')

In [14]:
data = pd.read_csv('../data/test_base.csv', encoding='utf-8')

In [ ]:
data = data[:100]

In [ ]:
from tqdm import tqdm
from datasets import Dataset, load_from_disk

gemma2

In [ ]:
ground_truths = data['Ответ сотрудника'].tolist()
answers = []
for question in tqdm(data['Вопрос пользователя']):
    prediction = chain.invoke(question)
    answers.append(prediction)


100%|██████████| 100/100 [13:16<00:00,  7.96s/it]


In [ ]:
from rouge import Rouge

rouge = Rouge()

scores = rouge.get_scores(answers, ground_truths, avg=True)

print("ROUGE-1: ", scores['rouge-1'])
print("ROUGE-2: ", scores['rouge-2'])
print("ROUGE-L: ", scores['rouge-l'])


ROUGE-1:  {'r': 0.4051708514051315, 'p': 0.40547657149093674, 'f': 0.3853999428010288}
ROUGE-2:  {'r': 0.3098056824820055, 'p': 0.31516324316045524, 'f': 0.29694316656142933}
ROUGE-L:  {'r': 0.3959456304136062, 'p': 0.39680840327476197, 'f': 0.3766770306941551}


С новым промтом

In [ ]:
from rouge import Rouge

rouge = Rouge()

scores = rouge.get_scores(answers, ground_truths, avg=True)

print("ROUGE-1: ", scores['rouge-1'])
print("ROUGE-2: ", scores['rouge-2'])
print("ROUGE-L: ", scores['rouge-l'])


ROUGE-1:  {'r': 0.42678218004526286, 'p': 0.4968356686994001, 'f': 0.4416919564269226}
ROUGE-2:  {'r': 0.37738458534996594, 'p': 0.4347684245874429, 'f': 0.3889902944121156}
ROUGE-L:  {'r': 0.42464681797143927, 'p': 0.49443860513085947, 'f': 0.4395035521254779}


\+ db top3

\+ rerank top2

\- bm

In [ ]:
from rouge import Rouge

rouge = Rouge()

scores = rouge.get_scores(answers, ground_truths, avg=True)

print("ROUGE-1: ", scores['rouge-1'])
print("ROUGE-2: ", scores['rouge-2'])
print("ROUGE-L: ", scores['rouge-l'])


ROUGE-1:  {'r': 0.4233416923949659, 'p': 0.5104342935278656, 'f': 0.4405773879511342}
ROUGE-2:  {'r': 0.3600688354165307, 'p': 0.42575274380276795, 'f': 0.3712909602238434}
ROUGE-L:  {'r': 0.41959057401862127, 'p': 0.5039508709562852, 'f': 0.4362472913632246}


На test_retriever без ансамбля (стало хуже)

In [ ]:
from rouge import Rouge

rouge = Rouge()

scores = rouge.get_scores(answers, ground_truths, avg=True)

print("ROUGE-1: ", scores['rouge-1'])
print("ROUGE-2: ", scores['rouge-2'])
print("ROUGE-L: ", scores['rouge-l'])


ROUGE-1:  {'r': 0.42930905747111564, 'p': 0.4909059867420198, 'f': 0.4313834346742969}
ROUGE-2:  {'r': 0.36751749465096184, 'p': 0.4200438286771497, 'f': 0.36923725257190376}
ROUGE-L:  {'r': 0.4240318496571432, 'p': 0.48482132821419, 'f': 0.42634277646289276}


Без бм 25

In [ ]:
from rouge import Rouge

rouge = Rouge()

scores = rouge.get_scores(answers, ground_truths, avg=True)

print("ROUGE-1: ", scores['rouge-1'])
print("ROUGE-2: ", scores['rouge-2'])
print("ROUGE-L: ", scores['rouge-l'])


ROUGE-1:  {'r': 0.4354718005843907, 'p': 0.5189136709034018, 'f': 0.45245438620460093}
ROUGE-2:  {'r': 0.36885349411062884, 'p': 0.440407938991423, 'f': 0.3837366774766806}
ROUGE-L:  {'r': 0.42836990725927754, 'p': 0.5117226677887935, 'f': 0.4455627967414783}


In [ ]:
data = pd.read_excel('../data/02_Реальные_кейсы.xlsx')

In [ ]:
data = data.drop(['Вопрос из БЗ', 'Классификатор 1 уровня', 'Классификатор 2 уровня'], axis=1)

In [ ]:
data.to_csv('../data/test_base_xl.csv', index=False, encoding='utf-8')

In [ ]:
data = pd.read_csv('../data/test_base_xl.csv', encoding='utf-8')

In [ ]:
ground_truths = data['Ответ сотрудника'].tolist()
answers = data['Ответ из БЗ'].tolist()

In [ ]:
rouge = Rouge()

scores = rouge.get_scores(answers, ground_truths, avg=True)

print("ROUGE-1: ", scores['rouge-1'])
print("ROUGE-2: ", scores['rouge-2'])
print("ROUGE-L: ", scores['rouge-l'])

ROUGE-1:  {'r': 0.6179374622517397, 'p': 0.6431418738503726, 'f': 0.6132308775817791}
ROUGE-2:  {'r': 0.5203859510429478, 'p': 0.5418607693118779, 'f': 0.5184162697290735}
ROUGE-L:  {'r': 0.6088197301852988, 'p': 0.6333266093416594, 'f': 0.6040425386299259}


In [ ]:
ground_truths = data['Ответ сотрудника'].tolist()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama.llms import OllamaLLM
from rouge import Rouge
from tqdm import tqdm

template = """
Ты интеллектуальный помощник компании rutube и помогаешь отвечать на вопросы. Будь вежливым.
Ответь на вопрос, используя следующий контекст, не меняя контекст и сохраняя все даты проведения событий, имена, аббревиатуры и ссылки:

{context}

Вопрос: {question}

Если не можешь найти ответ в контексте, вежливо скажи, что не знаешь ответ
"""
prompt = ChatPromptTemplate.from_template(template)

ollama_models = ['mistral:7b', 'wavecut/vikhr:latest', 'llama3.1:8b', 'llama3.2']

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

rouge = Rouge()

results = []

for model_name in ollama_models:
    print(f"Тестирование модели: {model_name}")

    model = OllamaLLM(model=model_name, base_url='http://26.251.162.207:11434')

    chain = (
        {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )

    answers = []

    for question in tqdm(data['Вопрос пользователя']):
        prediction = chain.invoke(question)
        answers.append(prediction)
    
    scores = rouge.get_scores(answers, ground_truths, avg=True)

    results.append({
        'model': model_name,
        'rouge-1': scores['rouge-1'],
        'rouge-2': scores['rouge-2'],
        'rouge-l': scores['rouge-l']
    })

for result in results:
    print(f"Модель: {result['model']}")
    print(f"ROUGE-1: {result['rouge-1']}")
    print(f"ROUGE-2: {result['rouge-2']}")
    print(f"ROUGE-L: {result['rouge-l']}")
    print()


Тестирование модели: mistral:7b


100%|██████████| 100/100 [13:24<00:00,  8.04s/it]


Тестирование модели: wavecut/vikhr:latest


100%|██████████| 100/100 [11:44<00:00,  7.05s/it]


Тестирование модели: llama3.1:8b


100%|██████████| 100/100 [12:43<00:00,  7.63s/it]


Тестирование модели: llama3.2


100%|██████████| 100/100 [10:24<00:00,  6.25s/it]


Модель: mistral:7b
ROUGE-1: {'r': 0.3566160023227822, 'p': 0.23847804953513235, 'f': 0.26199015733848524}
ROUGE-2: {'r': 0.20404036279759236, 'p': 0.1389016864550591, 'f': 0.14946098509926667}
ROUGE-L: {'r': 0.33413287030883077, 'p': 0.22282752766479302, 'f': 0.24475754765462096}

Модель: wavecut/vikhr:latest
ROUGE-1: {'r': 0.35277909808977825, 'p': 0.18864486188718355, 'f': 0.22483499787356467}
ROUGE-2: {'r': 0.14044921893020043, 'p': 0.0686982221584662, 'f': 0.08545386126331883}
ROUGE-L: {'r': 0.32998242362339836, 'p': 0.1749969200577814, 'f': 0.20876009944423352}

Модель: llama3.1:8b
ROUGE-1: {'r': 0.44601458170445235, 'p': 0.26387363182858875, 'f': 0.30756790134633744}
ROUGE-2: {'r': 0.3037452821847417, 'p': 0.17579250614584485, 'f': 0.20734890647935747}
ROUGE-L: {'r': 0.4231959136927327, 'p': 0.24981268412591845, 'f': 0.2915758313832965}

Модель: llama3.2
ROUGE-1: {'r': 0.3676250318097493, 'p': 0.1849241865042645, 'f': 0.21789556119966771}
ROUGE-2: {'r': 0.24297242819078843, 'p': 

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama.llms import OllamaLLM
from rouge import Rouge
from tqdm import tqdm

template = """
Ты интеллектуальный помощник компании rutube и помогаешь отвечать на вопросы. Будь вежливым.
Ответь на вопрос, используя следующий контекст, не меняя контекст и сохраняя все даты проведения событий, имена, аббревиатуры и ссылки:

{context}

Вопрос: {question}

Если не можешь найти ответ в контексте, вежливо скажи, что не знаешь ответ
"""
prompt = ChatPromptTemplate.from_template(template)

ollama_models = ['gemma2:9b-instruct-q6_K', 'qwen2.5:14b-instruct-q3_K_M']

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

rouge = Rouge()

results = []

for model_name in ollama_models:
    print(f"Тестирование модели: {model_name}")

    model = OllamaLLM(model=model_name, base_url='http://26.251.162.207:11434')

    chain = (
        {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )

    answers = []

    for question in tqdm(data['Вопрос пользователя']):
        prediction = chain.invoke(question)
        answers.append(prediction)
    
    scores = rouge.get_scores(answers, ground_truths, avg=True)

    results.append({
        'model': model_name,
        'rouge-1': scores['rouge-1'],
        'rouge-2': scores['rouge-2'],
        'rouge-l': scores['rouge-l']
    })

for result in results:
    print(f"Модель: {result['model']}")
    print(f"ROUGE-1: {result['rouge-1']}")
    print(f"ROUGE-2: {result['rouge-2']}")
    print(f"ROUGE-L: {result['rouge-l']}")
    print()


Тестирование модели: gemma2:9b-instruct-q6_K


100%|██████████| 100/100 [18:55<00:00, 11.35s/it]


Тестирование модели: qwen2.5:14b-instruct-q3_K_M


100%|██████████| 100/100 [32:12<00:00, 19.33s/it] 

Модель: gemma2:9b-instruct-q6_K
ROUGE-1: {'r': 0.37920031260262377, 'p': 0.4007276488136437, 'f': 0.3702487881195223}
ROUGE-2: {'r': 0.27628317848041295, 'p': 0.2896296727650362, 'f': 0.2676160295025228}
ROUGE-L: {'r': 0.3698622927147418, 'p': 0.39029766432865365, 'f': 0.36082769078950183}

Модель: qwen2.5:14b-instruct-q3_K_M
ROUGE-1: {'r': 0.3736570765553765, 'p': 0.24415983920974935, 'f': 0.2773864067984607}
ROUGE-2: {'r': 0.18449492169221401, 'p': 0.118165368278478, 'f': 0.1359410494926958}
ROUGE-L: {'r': 0.35611606801805407, 'p': 0.23209668862428387, 'f': 0.26380041501428875}

